In [2]:

import re
from sentence_transformers import SentenceTransformer


In [3]:
query = "the table has columns"

In [4]:

model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode(query, convert_to_tensor=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
import os
from pathlib import Path

def crawl_markdown_files(root_dir):
    """
    Crawls through directory and extracts text from markdown files.
    Returns a dict with filepath as key and content as value.
    """
    results = {}

    for path in Path(root_dir).rglob('*.md'):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                results[str(path)] = f.read()
        except Exception as e:
            print(f"Error reading {path}: {e}")

    return results

# Usage example:
# files = crawl_markdown_files('/path/to/folder')
# for path, content in files.items():
#     print(f"\n=== {path} ===\n{content[:100]}...")

crawled_result = crawl_markdown_files('workvault')
crawled_result

{'workvault/bbg vol surface.md': "\n- logic\n\t- in a nutshell, \n\t\t- implied forward and spot are right\n\t\t- but for discount factor, we will use our own\n\t\t- convert their volatility into adjusted volatility\n\n- I have an excel file on my desktop called `bbgvol`\n# Overrides\n\nthis is for the excel file we sent to Yiming\n\n- ==Rate set to BBGSOFRRate (just in excel)==\n- ==rename dividend to bbg dividend (in excel)==\n- ==price rename to bbg price (also just in excel)==\n\n## new explanation\nsplv internal\nspx bbg\n\nsplv gran\nsgx gran\nspx\n\n- cutoff date (recluse)\n- for splv, in recluse, we need to change from lv to the granular. we can also pass in the source (maybe not )... another cutoff date\n- spx, turn off old process and turn on the new one. We are ready and can do this\n\n\n- current plan is to, by the end of next week, do the deployment\n- Karthik... model change in month end\n- the third is the cutoff date\n\n\nkun's process uses cboe data and spx vol \n\n## 

In [6]:
crawled_result_embeddings = model.encode(list(crawled_result.values()), convert_to_tensor=True)
crawled_result_embeddings

tensor([[-0.0987, -0.0384, -0.0036,  ..., -0.0757, -0.0636,  0.0168],
        [-0.0548,  0.0196, -0.1599,  ..., -0.0499,  0.0533,  0.0237],
        [-0.0588, -0.0151, -0.0652,  ..., -0.0615, -0.0658,  0.0105],
        [ 0.0039, -0.0410, -0.0919,  ..., -0.0641, -0.1409,  0.0316],
        [-0.0406, -0.0418, -0.0468,  ..., -0.0347,  0.0147, -0.0380],
        [-0.1479, -0.0035, -0.0402,  ..., -0.0552,  0.0115, -0.0297]],
       device='cuda:0')

In [7]:
def split_sentences(text):
    # Split on ". " or ".\n" while keeping the period as part of the sentence
    sentences = re.split(r'(?<=[.?!])\s+|\n+', text.strip())
    return [s.strip() for s in sentences if s.strip()]

In [11]:

from annoy import AnnoyIndex
import random

f = 384  # Length of item vector that will be indexed

t = AnnoyIndex(f, 'angular')
for i, content_from_one_file in enumerate(crawled_result_embeddings):
    t.add_item(i, content_from_one_file)

t.build(10) # 10 trees
t.save('test.ann')

# ...

u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(1, 10)) # will find the 1000 nearest neighbors
# %%

[1, 3, 5, 0, 2, 4]


In [12]:
new_query_text = "the tables have columns"
new_query_embedding = model.encode(new_query_text, convert_to_tensor=True)

In [13]:
lookup_vec = new_query_embedding.unsqueeze(0)[0]
lookup_vec

tensor([ 9.2987e-02, -1.2371e-02, -9.6249e-02,  3.9771e-02, -1.5807e-02,
         2.4976e-02, -2.5786e-02, -9.6263e-03, -1.6224e-02,  5.5915e-02,
         8.2928e-02, -3.2445e-02,  2.3460e-02, -2.1310e-02, -1.0056e-01,
        -7.6212e-03, -8.4057e-02,  2.5401e-02, -4.4341e-02,  2.8322e-02,
        -7.6744e-02, -4.4769e-02, -8.5535e-03, -3.0795e-02,  3.7033e-02,
         9.9162e-02, -4.3898e-02,  8.3804e-02, -4.2000e-02, -7.9304e-02,
        -1.1251e-01,  2.0713e-02, -8.7833e-04,  5.8709e-02, -4.0790e-02,
        -8.7809e-02,  2.1477e-02,  4.0036e-02, -1.0505e-03,  8.2923e-03,
        -6.4526e-02, -1.4980e-01,  5.2970e-02,  3.1312e-02,  1.0414e-02,
         8.3096e-03, -7.0056e-02,  5.9505e-03, -1.9980e-03,  1.4767e-02,
        -8.1786e-02,  8.4996e-02, -1.0013e-02,  5.5677e-02,  9.2588e-03,
        -6.9061e-03, -6.1301e-02, -1.6839e-02,  1.4050e-02,  6.5332e-02,
        -8.9166e-02, -5.8575e-03,  2.8892e-02,  7.2687e-02, -1.0963e-02,
         1.3373e-02, -6.9836e-02,  3.4167e-02, -4.2

In [14]:

for ans_index in u.get_nns_by_vector(lookup_vec, 3):
  print(ans_index)

  # crawled_result_embeddings.values()[ans_index]

1
3
5


In [17]:
print(list(crawled_result.values())[1])


- what's referencing the retiring tables
	- ir options report, constants.py
		- in part 1... `class DbTables` and `GET_CASHFLOWS = dods.dbo.cashflows` or something like that... goes to `dods.dbo.cashflowtransactions`
	- payoff module
- the tables should be the same columns, so we might only have to change the name of the table
changes are in the following:
- https://securitybenefit.atlassian.net/browse/DB-4561
- https://securitybenefit.atlassian.net/browse/DB-4539
- https://securitybenefit.atlassian.net/browse/DB-4263

conversation in Teams

this seems to be working in 01Q


